In [1]:
# jupyter nbconvert --to python 6.bcs_tms_analysis.ipynb \
#     --TemplateExporter.exclude_markdown=True \
#     --TemplateExporter.exclude_output_prompt=True \
#     --TemplateExporter.exclude_input_prompt=True

In [2]:
import os
import sys
import gc
import ctypes
import re
import numpy as np
import pandas as pd
import platform
import logging
from datetime import datetime, timedelta
from pathlib import Path
import shutil
import zipfile
import duckdb 
import warnings
import fastparquet
from tqdm import tqdm 
from typing import List, Optional, Union
import psutil
import time # For timing the execution

warnings.filterwarnings('ignore')


# Optional: adjust pandas display for debugging; you can comment these out
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [3]:
def load_asc_lines(path):
    """
    Reads a Vector .asc CAN log file and extracts only CAN data lines.
    Returns a list of clean lines ready for parsing.
    """
    clean_lines = []
    with open(path, "r") as f:
        for ln in f:
            ln = ln.strip()
            if not ln:
                continue
            
            # CAN-frame lines always start with a float timestamp
            # Example: "0.000000 1 18f0090bx Rx d 8 ff ff ff 70 7d 74 7e 7d"
            parts = ln.split()
            if len(parts) >= 12:
                # Check if first token is a float timestamp
                try:
                    float(parts[0])
                    clean_lines.append(ln)
                except:
                    continue

    return clean_lines


In [4]:
import pandas as pd

def extract_cycle_cell_values(decoded, volt_cols, timestamp_col="timestamp_ist"):
    """
    Detects full cell-voltage cycles and extracts:
      - cycle_start, cycle_end
      - duration_sec, interval_sec
      - final Pack_cellVoltage_* values in each cycle

    Ignores Pack_cellVoltage_81–84 (never present).
    """

    # --- filter out dead cells 81–84 ---
    def get_cell_num(col):
        return int(col.split("_")[-1])

    # ignore_cells = {81, 82, 83, 84}
    valid_volt_cols = volt_cols
    # [
    #     c for c in volt_cols
    #     if get_cell_num(c) not in ignore_cells
    # ]
    cell_indices = [get_cell_num(c) for c in valid_volt_cols]
    num_cells = len(cell_indices)  # should be 572

    if num_cells == 0:
        print("⚠ No valid voltage columns found. Check volt_cols / column names.")
        return pd.DataFrame()

    print(f"Expecting {num_cells} cells per full cycle.")

    decoded = decoded.sort_values(timestamp_col).reset_index(drop=True)

    # cells present in each row
    decoded["cells_present"] = decoded[valid_volt_cols].notna().apply(
        lambda row: [cell_indices[i] for i, ok in enumerate(row) if ok],
        axis=1
    )

    # --- detect cycles ---
    cycles = []
    current_cells = set()
    cycle_start_time = None

    for _, row in decoded.iterrows():
        ts = row[timestamp_col]
        cells = row["cells_present"]

        if len(current_cells) == 0:
            cycle_start_time = ts

        current_cells.update(cells)

        if len(current_cells) == num_cells:
            cycles.append({"cycle_start": cycle_start_time, "cycle_end": ts})
            current_cells = set()

    # --- handle "no complete cycles" case cleanly ---
    if not cycles:
        print("⚠ No complete cycles detected in this dataset.")
        cols = ["cycle_start", "cycle_end", "duration_sec", "interval_sec"] + valid_volt_cols
        return pd.DataFrame(columns=cols)

    df_cycles = pd.DataFrame(cycles)
    df_cycles["duration_sec"] = (
        df_cycles["cycle_end"] - df_cycles["cycle_start"]
    ).dt.total_seconds()
    df_cycles["interval_sec"] = df_cycles["cycle_end"].diff().dt.total_seconds()
    df_cycles.loc[0, "interval_sec"] = None

    # --- extract final voltages for each cycle ---
    out_rows = []

    for _, cyc in df_cycles.iterrows():
        start, end = cyc["cycle_start"], cyc["cycle_end"]

        mask = (decoded[timestamp_col] >= start) & (decoded[timestamp_col] <= end)
        segment = decoded.loc[mask]

        if segment.empty:
            continue

        vals = segment[valid_volt_cols].ffill().bfill()
        if vals.empty:
            continue

        last_vals = vals.iloc[-1]

        row_out = {
            "cycle_start": start,
            "cycle_end": end,
            "duration_sec": cyc["duration_sec"],
            "interval_sec": cyc["interval_sec"],
        }

        for col in valid_volt_cols:
            row_out[col] = last_vals[col]

        out_rows.append(row_out)

    return pd.DataFrame(out_rows)


In [5]:
def convert_raw_can_lines(lines, bus_id=28, base_epoch_ms=None):
    rows = []
    seq = 0

    if base_epoch_ms is None:
        base_epoch_ms = int(time.time() * 1000)

    for ln in lines:
        ln = ln.strip()
        if not ln:
            continue

        parts = ln.split()
        
        rel_ts = float(parts[0])
        can_hex = parts[2][:-1]
        byte_vals = parts[-8:]

        # Create byte1...byte8 as integers
        bytes_dec = [int(b, 16) for b in byte_vals]

        ts_ms = base_epoch_ms + int(rel_ts * 1000)

        rows.append([
            bus_id,
            ts_ms,
            seq,
            int(can_hex,16),   # raw CAN ID
        ] + bytes_dec)

        seq += 1

    cols = ["id","timestamp","sequence","can_id"] + [f"byte{i}" for i in range(1,9)]
    return pd.DataFrame(rows, columns=cols)


In [6]:
dbc_lines = open("canId_conversions.txt").read().splitlines()

temp_map = {}   # dbc_id → list of 1–108
volt_map = {}   # dbc_id → list of 1–576

current_dbc_id = None
current_type = None  # "temp" or "volt"

temp_pattern = re.compile(r"SG_\s+Pack_Temperature(\d+)")
volt_pattern = re.compile(r"SG_\s+Pack_cellVoltage_(\d+)")

for line in dbc_lines:
    line = line.strip()

    # Detect BO_ lines
    if line.startswith("BO_ "):
        parts = line.split()
        current_dbc_id = int(parts[1])
        name = parts[2]

        if name.startswith("Pack_Temperature"):
            current_type = "temp"
            temp_map[current_dbc_id] = []

        elif name.startswith("Pack_cellVoltage"):
            current_type = "volt"
            volt_map[current_dbc_id] = []

        else:
            current_type = None  # ignore unrelated frames

        continue

    # Inside a BO_ block → extract signal names
    if current_type == "temp":
        m = temp_pattern.search(line)
        if m:
            idx = int(m.group(1))
            temp_map[current_dbc_id].append(idx)

    elif current_type == "volt":
        m = volt_pattern.search(line)
        if m:
            idx = int(m.group(1))
            volt_map[current_dbc_id].append(idx)


print("Temperature blocks:", len(temp_map))
print("Total temperature indices:", sum(len(v) for v in temp_map.values()))

print("Voltage blocks:", len(volt_map))
print("Total voltage indices:", sum(len(v) for v in volt_map.values()))


Temperature blocks: 15
Total temperature indices: 108
Voltage blocks: 145
Total voltage indices: 576


In [7]:
# asc_path = "can_Nov27_1300_10mins.asc"
asc_path = "../asc_csv_analyzer/nov26_6pm_1hr_id12.asc"
# asc_path = "can_20251027193535.asc"
# asc_path = "trip1.asc"
# asc_path = "debug_fb/can_20251027193535.asc"
raw_lines = load_asc_lines(asc_path)

df_asc = convert_raw_can_lines(
    raw_lines,
    bus_id=28,
    base_epoch_ms=1764229199971   # or any base timestamp you want
)

display(df_asc.head())

,id,timestamp,sequence,can_id,byte1,byte2,byte3,byte4,byte5,byte6,byte7,byte8
0,28,1764229199971,0,419373295,0,125,0,0,125,255,255,125
1,28,1764229199971,1,217449383,0,0,0,0,0,0,0,0
2,28,1764229199971,2,419373295,0,125,0,0,125,255,255,125
3,28,1764229199971,3,418579704,1,2,2,0,255,255,10,153
4,28,1764229199971,4,419358456,55,55,19,26,11,40,255,255


In [8]:
# df_asc[df_asc.can_id == 419343920]

In [9]:
df_asc.columns

Index(['id', 'timestamp', 'sequence', 'can_id', 'byte1', 'byte2', 'byte3',
       'byte4', 'byte5', 'byte6', 'byte7', 'byte8'],
      dtype='object')

In [10]:
df_map = pd.read_csv("dbc_to_raw_can_id_mapping.csv")
df_map = df_map.rename(columns={
    "dbc_can_id": "dbc_id",
    "raw_can_id": "raw_id"
})
df_map = df_map.rename(columns={
    "dbc_can_id": "dbc_id",
    "raw_can_id": "raw_id"
})
display(df_map.head())

# RAW → DBC (for decoding incoming CAN messages)
raw_to_dbc = dict(zip(df_map["raw_id"], df_map["dbc_id"]))

# DBC → RAW (if you ever need it)
dbc_to_raw = dict(zip(df_map["dbc_id"], df_map["raw_id"]))


,dbc_id,raw_id,dbc_hex,raw_hex
0,2556368883,408885235,0x985f17f3,0x185f17f3
1,2556434419,408950771,0x986017f3,0x186017f3
2,2556499955,409016307,0x986117f3,0x186117f3
3,2556565491,409081843,0x986217f3,0x186217f3
4,2556631027,409147379,0x986317f3,0x186317f3


In [11]:
df_asc['timestamp_ist'] = pd.to_datetime(df_asc['timestamp'], unit='ms', utc=True).dt.tz_convert('Asia/Kolkata').dt.tz_localize(None)
df_asc["dbc_id"] = df_asc["can_id"].map(raw_to_dbc)
df_asc.dbc_id = df_asc.dbc_id.astype("Int64")
df_asc[~df_asc["dbc_id"].isna()].head()

,id,timestamp,sequence,can_id,byte1,byte2,byte3,byte4,byte5,byte6,byte7,byte8,timestamp_ist,dbc_id
7,28,1764229199971,7,417732595,82,13,80,13,82,13,78,13,2025-11-27 13:09:59.971,2565216243
8,28,1764229199971,8,409343987,74,74,73,73,74,73,73,74,2025-11-27 13:09:59.971,2556827635
33,28,1764229200049,33,408885235,108,9,76,80,73,9,255,255,2025-11-27 13:10:00.049,2556368883
34,28,1764229200049,34,410982387,123,1,90,13,10,0,73,13,2025-11-27 13:10:00.049,2558466035
44,28,1764229200069,44,418518259,80,80,90,92,81,81,0,0,2025-11-27 13:10:00.069,2566001907


In [12]:
df_asc[df_asc.can_id == 2559186931].head()

,id,timestamp,sequence,can_id,byte1,byte2,byte3,byte4,byte5,byte6,byte7,byte8,timestamp_ist,dbc_id


In [13]:
decoded_asc = df_asc[["id","timestamp_ist","sequence"]].copy()

# Empty columns
for i in range(1, 577):
    decoded_asc[f"Pack_cellVoltage_{i}"] = None

for i in range(1, 109):
    decoded_asc[f"Pack_Temperature_{i}"] = None

# decoded_asc = df_asc[["id", "timestamp_ist", "sequence"]].copy()
# decoded_asc["payload"] = df_asc[byte_cols].values.tolist()
byte_cols = [f"byte{i}" for i in range(1,9)]
df_asc["payload"] = df_asc[byte_cols].values.tolist()
display(df_asc.head())

,id,timestamp,sequence,can_id,byte1,byte2,byte3,byte4,byte5,byte6,byte7,byte8,timestamp_ist,dbc_id,payload
0,28,1764229199971,0,419373295,0,125,0,0,125,255,255,125,2025-11-27 13:09:59.971,<NA>,"[0, 125, 0, 0, 125, 255, 255, 125]"
1,28,1764229199971,1,217449383,0,0,0,0,0,0,0,0,2025-11-27 13:09:59.971,<NA>,"[0, 0, 0, 0, 0, 0, 0, 0]"
2,28,1764229199971,2,419373295,0,125,0,0,125,255,255,125,2025-11-27 13:09:59.971,<NA>,"[0, 125, 0, 0, 125, 255, 255, 125]"
3,28,1764229199971,3,418579704,1,2,2,0,255,255,10,153,2025-11-27 13:09:59.971,<NA>,"[1, 2, 2, 0, 255, 255, 10, 153]"
4,28,1764229199971,4,419358456,55,55,19,26,11,40,255,255,2025-11-27 13:09:59.971,<NA>,"[55, 55, 19, 26, 11, 40, 255, 255]"


In [14]:
decoded_asc.head()

,id,timestamp_ist,sequence,Pack_cellVoltage_1,Pack_cellVoltage_2,Pack_cellVoltage_3,Pack_cellVoltage_4,Pack_cellVoltage_5,Pack_cellVoltage_6,Pack_cellVoltage_7,Pack_cellVoltage_8,Pack_cellVoltage_9,Pack_cellVoltage_10,Pack_cellVoltage_11,Pack_cellVoltage_12,Pack_cellVoltage_13,Pack_cellVoltage_14,Pack_cellVoltage_15,Pack_cellVoltage_16,Pack_cellVoltage_17,Pack_cellVoltage_18,Pack_cellVoltage_19,Pack_cellVoltage_20,Pack_cellVoltage_21,Pack_cellVoltage_22,Pack_cellVoltage_23,Pack_cellVoltage_24,Pack_cellVoltage_25,Pack_cellVoltage_26,Pack_cellVoltage_27,Pack_cellVoltage_28,Pack_cellVoltage_29,Pack_cellVoltage_30,Pack_cellVoltage_31,Pack_cellVoltage_32,Pack_cellVoltage_33,Pack_cellVoltage_34,Pack_cellVoltage_35,Pack_cellVoltage_36,Pack_cellVoltage_37,Pack_cellVoltage_38,Pack_cellVoltage_39,Pack_cellVoltage_40,Pack_cellVoltage_41,Pack_cellVoltage_42,Pack_cellVoltage_43,Pack_cellVoltage_44,Pack_cellVoltage_45,Pack_cellVoltage_46,Pack_cellVoltage_47,Pack_cellVoltage_48,Pack_cellVoltage_49,Pack_cellVoltage_50,Pack_cellVoltage_51,Pack_cellVoltage_52,Pack_cellVoltage_53,Pack_cellVoltage_54,Pack_cellVoltage_55,Pack_cellVoltage_56,Pack_cellVoltage_57,Pack_cellVoltage_58,Pack_cellVoltage_59,Pack_cellVoltage_60,Pack_cellVoltage_61,Pack_cellVoltage_62,Pack_cellVoltage_63,Pack_cellVoltage_64,Pack_cellVoltage_65,Pack_cellVoltage_66,Pack_cellVoltage_67,Pack_cellVoltage_68,Pack_cellVoltage_69,Pack_cellVoltage_70,Pack_cellVoltage_71,Pack_cellVoltage_72,Pack_cellVoltage_73,Pack_cellVoltage_74,Pack_cellVoltage_75,Pack_cellVoltage_76,Pack_cellVoltage_77,Pack_cellVoltage_78,Pack_cellVoltage_79,Pack_cellVoltage_80,Pack_cellVoltage_81,Pack_cellVoltage_82,Pack_cellVoltage_83,Pack_cellVoltage_84,Pack_cellVoltage_85,Pack_cellVoltage_86,Pack_cellVoltage_87,Pack_cellVoltage_88,Pack_cellVoltage_89,Pack_cellVoltage_90,Pack_cellVoltage_91,Pack_cellVoltage_92,Pack_cellVoltage_93,Pack_cellVoltage_94,Pack_cellVoltage_95,Pack_cellVoltage_96,Pack_cellVoltage_97,Pack_cellVoltage_98,Pack_cellVoltage_99,Pack_cellVoltage_100,Pack_cellVoltage_101,Pack_cellVoltage_102,Pack_cellVoltage_103,Pack_cellVoltage_104,Pack_cellVoltage_105,Pack_cellVoltage_106,Pack_cellVoltage_107,Pack_cellVoltage_108,Pack_cellVoltage_109,Pack_cellVoltage_110,Pack_cellVoltage_111,Pack_cellVoltage_112,Pack_cellVoltage_113,Pack_cellVoltage_114,Pack_cellVoltage_115,Pack_cellVoltage_116,Pack_cellVoltage_117,Pack_cellVoltage_118,Pack_cellVoltage_119,Pack_cellVoltage_120,Pack_cellVoltage_121,Pack_cellVoltage_122,Pack_cellVoltage_123,Pack_cellVoltage_124,Pack_cellVoltage_125,Pack_cellVoltage_126,Pack_cellVoltage_127,Pack_cellVoltage_128,Pack_cellVoltage_129,Pack_cellVoltage_130,Pack_cellVoltage_131,Pack_cellVoltage_132,Pack_cellVoltage_133,Pack_cellVoltage_134,Pack_cellVoltage_135,Pack_cellVoltage_136,Pack_cellVoltage_137,Pack_cellVoltage_138,Pack_cellVoltage_139,Pack_cellVoltage_140,Pack_cellVoltage_141,Pack_cellVoltage_142,Pack_cellVoltage_143,Pack_cellVoltage_144,Pack_cellVoltage_145,Pack_cellVoltage_146,Pack_cellVoltage_147,Pack_cellVoltage_148,Pack_cellVoltage_149,Pack_cellVoltage_150,Pack_cellVoltage_151,Pack_cellVoltage_152,Pack_cellVoltage_153,Pack_cellVoltage_154,Pack_cellVoltage_155,Pack_cellVoltage_156,Pack_cellVoltage_157,Pack_cellVoltage_158,Pack_cellVoltage_159,Pack_cellVoltage_160,Pack_cellVoltage_161,Pack_cellVoltage_162,Pack_cellVoltage_163,Pack_cellVoltage_164,Pack_cellVoltage_165,Pack_cellVoltage_166,Pack_cellVoltage_167,Pack_cellVoltage_168,Pack_cellVoltage_169,Pack_cellVoltage_170,Pack_cellVoltage_171,Pack_cellVoltage_172,Pack_cellVoltage_173,Pack_cellVoltage_174,Pack_cellVoltage_175,Pack_cellVoltage_176,Pack_cellVoltage_177,Pack_cellVoltage_178,Pack_cellVoltage_179,Pack_cellVoltage_180,Pack_cellVoltage_181,Pack_cellVoltage_182,Pack_cellVoltage_183,Pack_cellVoltage_184,Pack_cellVoltage_185,Pack_cellVoltage_186,Pack_cellVoltage_187,Pack_cellVoltage_188,Pack_cellVoltage_189,Pack_cellVoltage_190,Pack_cellVoltage_191,Pack_cellVoltage_192,Pack_cellVoltage_193,Pack_cellVoltage_194,Pack_ce

In [15]:
df_asc[~df_asc.dbc_id.isna()].dbc_id.sort_values().unique()

<IntegerArray>
[2556368883, 2556434419, 2556499955, 2556565491, 2556631027, 2556696563,
 2556762099, 2556827635, 2556893171, 2556958707,
 ...
 2567313395, 2567378931, 2567444467, 2567510003, 2567575539, 2567641075,
 2567706611, 2567772147, 2567837683, 2567903219]
Length: 157, dtype: Int64

In [16]:
for idx, row in df_asc.iterrows():
    dbc = row["dbc_id"]
    bytes_ = row["payload"]
    
    # Temperature frames
    if dbc in temp_map:
        sensor_indices = temp_map[dbc]
        for k, sensor_id in enumerate(sensor_indices):
            if sensor_id <= 108:
                decoded_asc.loc[idx, f"Pack_Temperature_{sensor_id}"] = bytes_[k] - 40

    # Voltage frames
    if dbc in volt_map:
        cells = volt_map[dbc]
        for k, cell_index in enumerate(cells):
            L = bytes_[k*2]       # lower byte
            H = bytes_[k*2 + 1]   # higher byte

            raw_val = (H << 8) | L
            decoded_value = raw_val * 0.001  # as per DBC scaling

            decoded_asc.loc[idx, f"Pack_cellVoltage_{cell_index}"] = decoded_value


In [17]:
decoded_asc[decoded_asc.Pack_cellVoltage_81.notna()].head()

,id,timestamp_ist,sequence,Pack_cellVoltage_1,Pack_cellVoltage_2,Pack_cellVoltage_3,Pack_cellVoltage_4,Pack_cellVoltage_5,Pack_cellVoltage_6,Pack_cellVoltage_7,Pack_cellVoltage_8,Pack_cellVoltage_9,Pack_cellVoltage_10,Pack_cellVoltage_11,Pack_cellVoltage_12,Pack_cellVoltage_13,Pack_cellVoltage_14,Pack_cellVoltage_15,Pack_cellVoltage_16,Pack_cellVoltage_17,Pack_cellVoltage_18,Pack_cellVoltage_19,Pack_cellVoltage_20,Pack_cellVoltage_21,Pack_cellVoltage_22,Pack_cellVoltage_23,Pack_cellVoltage_24,Pack_cellVoltage_25,Pack_cellVoltage_26,Pack_cellVoltage_27,Pack_cellVoltage_28,Pack_cellVoltage_29,Pack_cellVoltage_30,Pack_cellVoltage_31,Pack_cellVoltage_32,Pack_cellVoltage_33,Pack_cellVoltage_34,Pack_cellVoltage_35,Pack_cellVoltage_36,Pack_cellVoltage_37,Pack_cellVoltage_38,Pack_cellVoltage_39,Pack_cellVoltage_40,Pack_cellVoltage_41,Pack_cellVoltage_42,Pack_cellVoltage_43,Pack_cellVoltage_44,Pack_cellVoltage_45,Pack_cellVoltage_46,Pack_cellVoltage_47,Pack_cellVoltage_48,Pack_cellVoltage_49,Pack_cellVoltage_50,Pack_cellVoltage_51,Pack_cellVoltage_52,Pack_cellVoltage_53,Pack_cellVoltage_54,Pack_cellVoltage_55,Pack_cellVoltage_56,Pack_cellVoltage_57,Pack_cellVoltage_58,Pack_cellVoltage_59,Pack_cellVoltage_60,Pack_cellVoltage_61,Pack_cellVoltage_62,Pack_cellVoltage_63,Pack_cellVoltage_64,Pack_cellVoltage_65,Pack_cellVoltage_66,Pack_cellVoltage_67,Pack_cellVoltage_68,Pack_cellVoltage_69,Pack_cellVoltage_70,Pack_cellVoltage_71,Pack_cellVoltage_72,Pack_cellVoltage_73,Pack_cellVoltage_74,Pack_cellVoltage_75,Pack_cellVoltage_76,Pack_cellVoltage_77,Pack_cellVoltage_78,Pack_cellVoltage_79,Pack_cellVoltage_80,Pack_cellVoltage_81,Pack_cellVoltage_82,Pack_cellVoltage_83,Pack_cellVoltage_84,Pack_cellVoltage_85,Pack_cellVoltage_86,Pack_cellVoltage_87,Pack_cellVoltage_88,Pack_cellVoltage_89,Pack_cellVoltage_90,Pack_cellVoltage_91,Pack_cellVoltage_92,Pack_cellVoltage_93,Pack_cellVoltage_94,Pack_cellVoltage_95,Pack_cellVoltage_96,Pack_cellVoltage_97,Pack_cellVoltage_98,Pack_cellVoltage_99,Pack_cellVoltage_100,Pack_cellVoltage_101,Pack_cellVoltage_102,Pack_cellVoltage_103,Pack_cellVoltage_104,Pack_cellVoltage_105,Pack_cellVoltage_106,Pack_cellVoltage_107,Pack_cellVoltage_108,Pack_cellVoltage_109,Pack_cellVoltage_110,Pack_cellVoltage_111,Pack_cellVoltage_112,Pack_cellVoltage_113,Pack_cellVoltage_114,Pack_cellVoltage_115,Pack_cellVoltage_116,Pack_cellVoltage_117,Pack_cellVoltage_118,Pack_cellVoltage_119,Pack_cellVoltage_120,Pack_cellVoltage_121,Pack_cellVoltage_122,Pack_cellVoltage_123,Pack_cellVoltage_124,Pack_cellVoltage_125,Pack_cellVoltage_126,Pack_cellVoltage_127,Pack_cellVoltage_128,Pack_cellVoltage_129,Pack_cellVoltage_130,Pack_cellVoltage_131,Pack_cellVoltage_132,Pack_cellVoltage_133,Pack_cellVoltage_134,Pack_cellVoltage_135,Pack_cellVoltage_136,Pack_cellVoltage_137,Pack_cellVoltage_138,Pack_cellVoltage_139,Pack_cellVoltage_140,Pack_cellVoltage_141,Pack_cellVoltage_142,Pack_cellVoltage_143,Pack_cellVoltage_144,Pack_cellVoltage_145,Pack_cellVoltage_146,Pack_cellVoltage_147,Pack_cellVoltage_148,Pack_cellVoltage_149,Pack_cellVoltage_150,Pack_cellVoltage_151,Pack_cellVoltage_152,Pack_cellVoltage_153,Pack_cellVoltage_154,Pack_cellVoltage_155,Pack_cellVoltage_156,Pack_cellVoltage_157,Pack_cellVoltage_158,Pack_cellVoltage_159,Pack_cellVoltage_160,Pack_cellVoltage_161,Pack_cellVoltage_162,Pack_cellVoltage_163,Pack_cellVoltage_164,Pack_cellVoltage_165,Pack_cellVoltage_166,Pack_cellVoltage_167,Pack_cellVoltage_168,Pack_cellVoltage_169,Pack_cellVoltage_170,Pack_cellVoltage_171,Pack_cellVoltage_172,Pack_cellVoltage_173,Pack_cellVoltage_174,Pack_cellVoltage_175,Pack_cellVoltage_176,Pack_cellVoltage_177,Pack_cellVoltage_178,Pack_cellVoltage_179,Pack_cellVoltage_180,Pack_cellVoltage_181,Pack_cellVoltage_182,Pack_cellVoltage_183,Pack_cellVoltage_184,Pack_cellVoltage_185,Pack_cellVoltage_186,Pack_cellVoltage_187,Pack_cellVoltage_188,Pack_cellVoltage_189,Pack_cellVoltage_190,Pack_cellVoltage_191,Pack_cellVoltage_192,Pack_cellVoltage_193,Pack_cellVoltage_194,Pack_ce

In [18]:
decoded_asc[decoded_asc.Pack_cellVoltage_1.notna()|decoded_asc.Pack_cellVoltage_5.notna()|decoded_asc.Pack_cellVoltage_9.notna()].head()

,id,timestamp_ist,sequence,Pack_cellVoltage_1,Pack_cellVoltage_2,Pack_cellVoltage_3,Pack_cellVoltage_4,Pack_cellVoltage_5,Pack_cellVoltage_6,Pack_cellVoltage_7,Pack_cellVoltage_8,Pack_cellVoltage_9,Pack_cellVoltage_10,Pack_cellVoltage_11,Pack_cellVoltage_12,Pack_cellVoltage_13,Pack_cellVoltage_14,Pack_cellVoltage_15,Pack_cellVoltage_16,Pack_cellVoltage_17,Pack_cellVoltage_18,Pack_cellVoltage_19,Pack_cellVoltage_20,Pack_cellVoltage_21,Pack_cellVoltage_22,Pack_cellVoltage_23,Pack_cellVoltage_24,Pack_cellVoltage_25,Pack_cellVoltage_26,Pack_cellVoltage_27,Pack_cellVoltage_28,Pack_cellVoltage_29,Pack_cellVoltage_30,Pack_cellVoltage_31,Pack_cellVoltage_32,Pack_cellVoltage_33,Pack_cellVoltage_34,Pack_cellVoltage_35,Pack_cellVoltage_36,Pack_cellVoltage_37,Pack_cellVoltage_38,Pack_cellVoltage_39,Pack_cellVoltage_40,Pack_cellVoltage_41,Pack_cellVoltage_42,Pack_cellVoltage_43,Pack_cellVoltage_44,Pack_cellVoltage_45,Pack_cellVoltage_46,Pack_cellVoltage_47,Pack_cellVoltage_48,Pack_cellVoltage_49,Pack_cellVoltage_50,Pack_cellVoltage_51,Pack_cellVoltage_52,Pack_cellVoltage_53,Pack_cellVoltage_54,Pack_cellVoltage_55,Pack_cellVoltage_56,Pack_cellVoltage_57,Pack_cellVoltage_58,Pack_cellVoltage_59,Pack_cellVoltage_60,Pack_cellVoltage_61,Pack_cellVoltage_62,Pack_cellVoltage_63,Pack_cellVoltage_64,Pack_cellVoltage_65,Pack_cellVoltage_66,Pack_cellVoltage_67,Pack_cellVoltage_68,Pack_cellVoltage_69,Pack_cellVoltage_70,Pack_cellVoltage_71,Pack_cellVoltage_72,Pack_cellVoltage_73,Pack_cellVoltage_74,Pack_cellVoltage_75,Pack_cellVoltage_76,Pack_cellVoltage_77,Pack_cellVoltage_78,Pack_cellVoltage_79,Pack_cellVoltage_80,Pack_cellVoltage_81,Pack_cellVoltage_82,Pack_cellVoltage_83,Pack_cellVoltage_84,Pack_cellVoltage_85,Pack_cellVoltage_86,Pack_cellVoltage_87,Pack_cellVoltage_88,Pack_cellVoltage_89,Pack_cellVoltage_90,Pack_cellVoltage_91,Pack_cellVoltage_92,Pack_cellVoltage_93,Pack_cellVoltage_94,Pack_cellVoltage_95,Pack_cellVoltage_96,Pack_cellVoltage_97,Pack_cellVoltage_98,Pack_cellVoltage_99,Pack_cellVoltage_100,Pack_cellVoltage_101,Pack_cellVoltage_102,Pack_cellVoltage_103,Pack_cellVoltage_104,Pack_cellVoltage_105,Pack_cellVoltage_106,Pack_cellVoltage_107,Pack_cellVoltage_108,Pack_cellVoltage_109,Pack_cellVoltage_110,Pack_cellVoltage_111,Pack_cellVoltage_112,Pack_cellVoltage_113,Pack_cellVoltage_114,Pack_cellVoltage_115,Pack_cellVoltage_116,Pack_cellVoltage_117,Pack_cellVoltage_118,Pack_cellVoltage_119,Pack_cellVoltage_120,Pack_cellVoltage_121,Pack_cellVoltage_122,Pack_cellVoltage_123,Pack_cellVoltage_124,Pack_cellVoltage_125,Pack_cellVoltage_126,Pack_cellVoltage_127,Pack_cellVoltage_128,Pack_cellVoltage_129,Pack_cellVoltage_130,Pack_cellVoltage_131,Pack_cellVoltage_132,Pack_cellVoltage_133,Pack_cellVoltage_134,Pack_cellVoltage_135,Pack_cellVoltage_136,Pack_cellVoltage_137,Pack_cellVoltage_138,Pack_cellVoltage_139,Pack_cellVoltage_140,Pack_cellVoltage_141,Pack_cellVoltage_142,Pack_cellVoltage_143,Pack_cellVoltage_144,Pack_cellVoltage_145,Pack_cellVoltage_146,Pack_cellVoltage_147,Pack_cellVoltage_148,Pack_cellVoltage_149,Pack_cellVoltage_150,Pack_cellVoltage_151,Pack_cellVoltage_152,Pack_cellVoltage_153,Pack_cellVoltage_154,Pack_cellVoltage_155,Pack_cellVoltage_156,Pack_cellVoltage_157,Pack_cellVoltage_158,Pack_cellVoltage_159,Pack_cellVoltage_160,Pack_cellVoltage_161,Pack_cellVoltage_162,Pack_cellVoltage_163,Pack_cellVoltage_164,Pack_cellVoltage_165,Pack_cellVoltage_166,Pack_cellVoltage_167,Pack_cellVoltage_168,Pack_cellVoltage_169,Pack_cellVoltage_170,Pack_cellVoltage_171,Pack_cellVoltage_172,Pack_cellVoltage_173,Pack_cellVoltage_174,Pack_cellVoltage_175,Pack_cellVoltage_176,Pack_cellVoltage_177,Pack_cellVoltage_178,Pack_cellVoltage_179,Pack_cellVoltage_180,Pack_cellVoltage_181,Pack_cellVoltage_182,Pack_cellVoltage_183,Pack_cellVoltage_184,Pack_cellVoltage_185,Pack_cellVoltage_186,Pack_cellVoltage_187,Pack_cellVoltage_188,Pack_cellVoltage_189,Pack_cellVoltage_190,Pack_cellVoltage_191,Pack_cellVoltage_192,Pack_cellVoltage_193,Pack_cellVoltage_194,Pack_ce

In [19]:
decoded_asc[decoded_asc.Pack_Temperature_72.notna()].head()

,id,timestamp_ist,sequence,Pack_cellVoltage_1,Pack_cellVoltage_2,Pack_cellVoltage_3,Pack_cellVoltage_4,Pack_cellVoltage_5,Pack_cellVoltage_6,Pack_cellVoltage_7,Pack_cellVoltage_8,Pack_cellVoltage_9,Pack_cellVoltage_10,Pack_cellVoltage_11,Pack_cellVoltage_12,Pack_cellVoltage_13,Pack_cellVoltage_14,Pack_cellVoltage_15,Pack_cellVoltage_16,Pack_cellVoltage_17,Pack_cellVoltage_18,Pack_cellVoltage_19,Pack_cellVoltage_20,Pack_cellVoltage_21,Pack_cellVoltage_22,Pack_cellVoltage_23,Pack_cellVoltage_24,Pack_cellVoltage_25,Pack_cellVoltage_26,Pack_cellVoltage_27,Pack_cellVoltage_28,Pack_cellVoltage_29,Pack_cellVoltage_30,Pack_cellVoltage_31,Pack_cellVoltage_32,Pack_cellVoltage_33,Pack_cellVoltage_34,Pack_cellVoltage_35,Pack_cellVoltage_36,Pack_cellVoltage_37,Pack_cellVoltage_38,Pack_cellVoltage_39,Pack_cellVoltage_40,Pack_cellVoltage_41,Pack_cellVoltage_42,Pack_cellVoltage_43,Pack_cellVoltage_44,Pack_cellVoltage_45,Pack_cellVoltage_46,Pack_cellVoltage_47,Pack_cellVoltage_48,Pack_cellVoltage_49,Pack_cellVoltage_50,Pack_cellVoltage_51,Pack_cellVoltage_52,Pack_cellVoltage_53,Pack_cellVoltage_54,Pack_cellVoltage_55,Pack_cellVoltage_56,Pack_cellVoltage_57,Pack_cellVoltage_58,Pack_cellVoltage_59,Pack_cellVoltage_60,Pack_cellVoltage_61,Pack_cellVoltage_62,Pack_cellVoltage_63,Pack_cellVoltage_64,Pack_cellVoltage_65,Pack_cellVoltage_66,Pack_cellVoltage_67,Pack_cellVoltage_68,Pack_cellVoltage_69,Pack_cellVoltage_70,Pack_cellVoltage_71,Pack_cellVoltage_72,Pack_cellVoltage_73,Pack_cellVoltage_74,Pack_cellVoltage_75,Pack_cellVoltage_76,Pack_cellVoltage_77,Pack_cellVoltage_78,Pack_cellVoltage_79,Pack_cellVoltage_80,Pack_cellVoltage_81,Pack_cellVoltage_82,Pack_cellVoltage_83,Pack_cellVoltage_84,Pack_cellVoltage_85,Pack_cellVoltage_86,Pack_cellVoltage_87,Pack_cellVoltage_88,Pack_cellVoltage_89,Pack_cellVoltage_90,Pack_cellVoltage_91,Pack_cellVoltage_92,Pack_cellVoltage_93,Pack_cellVoltage_94,Pack_cellVoltage_95,Pack_cellVoltage_96,Pack_cellVoltage_97,Pack_cellVoltage_98,Pack_cellVoltage_99,Pack_cellVoltage_100,Pack_cellVoltage_101,Pack_cellVoltage_102,Pack_cellVoltage_103,Pack_cellVoltage_104,Pack_cellVoltage_105,Pack_cellVoltage_106,Pack_cellVoltage_107,Pack_cellVoltage_108,Pack_cellVoltage_109,Pack_cellVoltage_110,Pack_cellVoltage_111,Pack_cellVoltage_112,Pack_cellVoltage_113,Pack_cellVoltage_114,Pack_cellVoltage_115,Pack_cellVoltage_116,Pack_cellVoltage_117,Pack_cellVoltage_118,Pack_cellVoltage_119,Pack_cellVoltage_120,Pack_cellVoltage_121,Pack_cellVoltage_122,Pack_cellVoltage_123,Pack_cellVoltage_124,Pack_cellVoltage_125,Pack_cellVoltage_126,Pack_cellVoltage_127,Pack_cellVoltage_128,Pack_cellVoltage_129,Pack_cellVoltage_130,Pack_cellVoltage_131,Pack_cellVoltage_132,Pack_cellVoltage_133,Pack_cellVoltage_134,Pack_cellVoltage_135,Pack_cellVoltage_136,Pack_cellVoltage_137,Pack_cellVoltage_138,Pack_cellVoltage_139,Pack_cellVoltage_140,Pack_cellVoltage_141,Pack_cellVoltage_142,Pack_cellVoltage_143,Pack_cellVoltage_144,Pack_cellVoltage_145,Pack_cellVoltage_146,Pack_cellVoltage_147,Pack_cellVoltage_148,Pack_cellVoltage_149,Pack_cellVoltage_150,Pack_cellVoltage_151,Pack_cellVoltage_152,Pack_cellVoltage_153,Pack_cellVoltage_154,Pack_cellVoltage_155,Pack_cellVoltage_156,Pack_cellVoltage_157,Pack_cellVoltage_158,Pack_cellVoltage_159,Pack_cellVoltage_160,Pack_cellVoltage_161,Pack_cellVoltage_162,Pack_cellVoltage_163,Pack_cellVoltage_164,Pack_cellVoltage_165,Pack_cellVoltage_166,Pack_cellVoltage_167,Pack_cellVoltage_168,Pack_cellVoltage_169,Pack_cellVoltage_170,Pack_cellVoltage_171,Pack_cellVoltage_172,Pack_cellVoltage_173,Pack_cellVoltage_174,Pack_cellVoltage_175,Pack_cellVoltage_176,Pack_cellVoltage_177,Pack_cellVoltage_178,Pack_cellVoltage_179,Pack_cellVoltage_180,Pack_cellVoltage_181,Pack_cellVoltage_182,Pack_cellVoltage_183,Pack_cellVoltage_184,Pack_cellVoltage_185,Pack_cellVoltage_186,Pack_cellVoltage_187,Pack_cellVoltage_188,Pack_cellVoltage_189,Pack_cellVoltage_190,Pack_cellVoltage_191,Pack_cellVoltage_192,Pack_cellVoltage_193,Pack_cellVoltage_194,Pack_ce

In [20]:
# Get all cell voltage columns
volt_cols = [c for c in decoded_asc.columns if c.startswith("Pack_cellVoltage_")]

# Extract numeric index from each column
cell_indices = [int(c.split("_")[-1]) for c in volt_cols]
num_cells = len(cell_indices)  # should be 576

decoded_asc["cells_present"] = decoded_asc[volt_cols].notna().apply(lambda row: 
    [cell_indices[i] for i, present in enumerate(row) if present], axis=1)

cycles = []
current_cells = set()
cycle_start_time = None

for idx, row in decoded_asc.iterrows():
    cells = row["cells_present"]
    ts = row["timestamp_ist"]

    # Start of a new cycle
    if len(current_cells) == 0:
        cycle_start_time = ts

    # Add newly seen cells
    current_cells.update(cells)

    # If full cycle achieved
    if len(current_cells) == num_cells:
        cycle_end_time = ts
        duration = (cycle_end_time - cycle_start_time).total_seconds()

        cycles.append({
            "cycle_start": cycle_start_time,
            "cycle_end": cycle_end_time,
            "duration_sec": duration
        })

        # Reset for next cycle
        current_cells = set()

df_cycles = pd.DataFrame(cycles)
display(df_cycles)


,cycle_start,cycle_end,duration_sec
0,2025-11-27 13:09:59.971,2025-11-27 13:11:34.889,94.918
1,2025-11-27 13:11:35.092,2025-11-27 13:12:29.229,54.137
2,2025-11-27 13:12:29.429,2025-11-27 13:13:31.662,62.233
3,2025-11-27 13:13:31.807,2025-11-27 13:14:55.271,83.464
4,2025-11-27 13:14:55.411,2025-11-27 13:15:28.831,33.420
5,2025-11-27 13:15:29.011,2025-11-27 13:16:47.051,78.040
6,2025-11-27 13:16:47.251,2025-11-27 13:18:08.471,81.220
7,2025-11-27 13:18:08.891,2025-11-27 13:19:58.411,109.520
8,2025-11-27 13:19:58.631,2025-11-27 13:20:58.394,59.763
9,2025-11-27 13:20:58.591,2025-11-27 13:22:04.091,65.500


In [21]:
volt_cols = [c for c in decoded_asc.columns if c.startswith("Pack_cellVoltage_")]

df_asc_cycle_voltages = extract_cycle_cell_values(decoded_asc, volt_cols)
    
display(df_asc_cycle_voltages)

Expecting 576 cells per full cycle.


,cycle_start,cycle_end,duration_sec,interval_sec,Pack_cellVoltage_1,Pack_cellVoltage_2,Pack_cellVoltage_3,Pack_cellVoltage_4,Pack_cellVoltage_5,Pack_cellVoltage_6,Pack_cellVoltage_7,Pack_cellVoltage_8,Pack_cellVoltage_9,Pack_cellVoltage_10,Pack_cellVoltage_11,Pack_cellVoltage_12,Pack_cellVoltage_13,Pack_cellVoltage_14,Pack_cellVoltage_15,Pack_cellVoltage_16,Pack_cellVoltage_17,Pack_cellVoltage_18,Pack_cellVoltage_19,Pack_cellVoltage_20,Pack_cellVoltage_21,Pack_cellVoltage_22,Pack_cellVoltage_23,Pack_cellVoltage_24,Pack_cellVoltage_25,Pack_cellVoltage_26,Pack_cellVoltage_27,Pack_cellVoltage_28,Pack_cellVoltage_29,Pack_cellVoltage_30,Pack_cellVoltage_31,Pack_cellVoltage_32,Pack_cellVoltage_33,Pack_cellVoltage_34,Pack_cellVoltage_35,Pack_cellVoltage_36,Pack_cellVoltage_37,Pack_cellVoltage_38,Pack_cellVoltage_39,Pack_cellVoltage_40,Pack_cellVoltage_41,Pack_cellVoltage_42,Pack_cellVoltage_43,Pack_cellVoltage_44,Pack_cellVoltage_45,Pack_cellVoltage_46,Pack_cellVoltage_47,Pack_cellVoltage_48,Pack_cellVoltage_49,Pack_cellVoltage_50,Pack_cellVoltage_51,Pack_cellVoltage_52,Pack_cellVoltage_53,Pack_cellVoltage_54,Pack_cellVoltage_55,Pack_cellVoltage_56,Pack_cellVoltage_57,Pack_cellVoltage_58,Pack_cellVoltage_59,Pack_cellVoltage_60,Pack_cellVoltage_61,Pack_cellVoltage_62,Pack_cellVoltage_63,Pack_cellVoltage_64,Pack_cellVoltage_65,Pack_cellVoltage_66,Pack_cellVoltage_67,Pack_cellVoltage_68,Pack_cellVoltage_69,Pack_cellVoltage_70,Pack_cellVoltage_71,Pack_cellVoltage_72,Pack_cellVoltage_73,Pack_cellVoltage_74,Pack_cellVoltage_75,Pack_cellVoltage_76,Pack_cellVoltage_77,Pack_cellVoltage_78,Pack_cellVoltage_79,Pack_cellVoltage_80,Pack_cellVoltage_81,Pack_cellVoltage_82,Pack_cellVoltage_83,Pack_cellVoltage_84,Pack_cellVoltage_85,Pack_cellVoltage_86,Pack_cellVoltage_87,Pack_cellVoltage_88,Pack_cellVoltage_89,Pack_cellVoltage_90,Pack_cellVoltage_91,Pack_cellVoltage_92,Pack_cellVoltage_93,Pack_cellVoltage_94,Pack_cellVoltage_95,Pack_cellVoltage_96,Pack_cellVoltage_97,Pack_cellVoltage_98,Pack_cellVoltage_99,Pack_cellVoltage_100,Pack_cellVoltage_101,Pack_cellVoltage_102,Pack_cellVoltage_103,Pack_cellVoltage_104,Pack_cellVoltage_105,Pack_cellVoltage_106,Pack_cellVoltage_107,Pack_cellVoltage_108,Pack_cellVoltage_109,Pack_cellVoltage_110,Pack_cellVoltage_111,Pack_cellVoltage_112,Pack_cellVoltage_113,Pack_cellVoltage_114,Pack_cellVoltage_115,Pack_cellVoltage_116,Pack_cellVoltage_117,Pack_cellVoltage_118,Pack_cellVoltage_119,Pack_cellVoltage_120,Pack_cellVoltage_121,Pack_cellVoltage_122,Pack_cellVoltage_123,Pack_cellVoltage_124,Pack_cellVoltage_125,Pack_cellVoltage_126,Pack_cellVoltage_127,Pack_cellVoltage_128,Pack_cellVoltage_129,Pack_cellVoltage_130,Pack_cellVoltage_131,Pack_cellVoltage_132,Pack_cellVoltage_133,Pack_cellVoltage_134,Pack_cellVoltage_135,Pack_cellVoltage_136,Pack_cellVoltage_137,Pack_cellVoltage_138,Pack_cellVoltage_139,Pack_cellVoltage_140,Pack_cellVoltage_141,Pack_cellVoltage_142,Pack_cellVoltage_143,Pack_cellVoltage_144,Pack_cellVoltage_145,Pack_cellVoltage_146,Pack_cellVoltage_147,Pack_cellVoltage_148,Pack_cellVoltage_149,Pack_cellVoltage_150,Pack_cellVoltage_151,Pack_cellVoltage_152,Pack_cellVoltage_153,Pack_cellVoltage_154,Pack_cellVoltage_155,Pack_cellVoltage_156,Pack_cellVoltage_157,Pack_cellVoltage_158,Pack_cellVoltage_159,Pack_cellVoltage_160,Pack_cellVoltage_161,Pack_cellVoltage_162,Pack_cellVoltage_163,Pack_cellVoltage_164,Pack_cellVoltage_165,Pack_cellVoltage_166,Pack_cellVoltage_167,Pack_cellVoltage_168,Pack_cellVoltage_169,Pack_cellVoltage_170,Pack_cellVoltage_171,Pack_cellVoltage_172,Pack_cellVoltage_173,Pack_cellVoltage_174,Pack_cellVoltage_175,Pack_cellVoltage_176,Pack_cellVoltage_177,Pack_cellVoltage_178,Pack_cellVoltage_179,Pack_cellVoltage_180,Pack_cellVoltage_181,Pack_cellVoltage_182,Pack_cellVoltage_183,Pack_cellVoltage_184,Pack_cellVoltage_185,Pack_cellVoltage_186,Pack_cellVoltage_187,Pack_cellVoltage_188,Pack_cellVoltage_189,Pack_cellVoltage_190,Pack_cellVoltage_191,Pack_cellVoltage_192,Pack_cellVoltage_193,Pack_c

In [22]:
df_asc_cycle_voltages.to_csv("df_asc_cycle_cell_voltages.csv", index=False)

In [23]:
df_asc_cycle_voltages.duration_sec.describe(percentiles=[0.25,0.5,0.75,0.9,0.95,0.99])

count     33.000000
mean      66.404061
std       20.744029
min       28.675000
25%       54.080000
50%       62.233000
75%       81.840000
90%       93.102400
95%      104.506000
99%      109.257600
max      109.520000
Name: duration_sec, dtype: float64